# Combat Simulator Comparison v2

Setup-aware comparison of `sc2helper` (ground truth) vs numpy Lanchester simulator.

In [1]:
from __future__ import annotations

import lzma
import pickle
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import r2_score

ROOT = Path.cwd()
if not (ROOT / "phantom").exists():
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from phantom.mock.combat_sim import generate_mock_combat_dataset
from phantom.mock.hp_ratio_sim import predict_outcome as hp_ratio_predict_outcome

DATASET_PATH = ROOT / "resources/combat_sim/mock.pkl.xz"
RUN_DIRECT = True
SIMULATION_COUNT = 1000
SPAWN_COUNT = 10
SEED = 1337


In [2]:
if RUN_DIRECT or not DATASET_PATH.exists():
    records = generate_mock_combat_dataset(
        simulation_count=SIMULATION_COUNT,
        spawn_count=SPAWN_COUNT,
        use_position=True,
        seed=SEED,
    )
    source = "direct"
else:
    with lzma.open(DATASET_PATH, "rb") as f:
        records = pickle.load(f)
    source = str(DATASET_PATH)

df = pd.DataFrame(records)
if "pred_outcome_hp_ratio" not in df.columns:
    def _hp_ratio_from_units(units: list[dict]) -> float:
        units1 = [u for u in units if not bool(u["is_enemy"])]
        units2 = [u for u in units if bool(u["is_enemy"])]
        return hp_ratio_predict_outcome(units1, units2)

    df["pred_outcome_hp_ratio"] = df["units"].map(_hp_ratio_from_units)
required = {"setup", "parameter_name", "parameter_value", "true_outcome", "pred_outcome"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"dataset missing setup-comparison columns: {sorted(missing)}")

df = df.sort_values(["setup", "parameter_value"]).reset_index(drop=True)
print(f"Loaded {len(df):,} rows from {source}")
df.head(3)


Combat simulations (mock setups): 100%|██████████| 1000/1000 [00:02<00:00, 437.40it/s]

Loaded 1,000 rows from direct


,setup,parameter_name,parameter_value,units,true_outcome,pred_outcome,pred_outcome_hp_ratio,true_advantage_log,true_bitterness_log
0,circle,radius,2.0,"[{'tag': 1, 'is_enemy': False, 'is_flying': Fa...",-0.472779,-0.045160,-0.020223,-0.320067,-0.320067
1,circle,radius,2.0,"[{'tag': 1, 'is_enemy': False, 'is_flying': Fa...",-0.397516,0.089533,0.051829,-0.253347,-0.253347
2,circle,radius,2.0,"[{'tag': 1, 'is_enemy': False, 'is_flying': Fa...",-0.251960,0.054713,0.031325,-0.145149,-0.145149


In [3]:
setups = sorted(df["setup"].unique())
fig = make_subplots(rows=1, cols=len(setups), subplot_titles=setups)

for col, setup in enumerate(setups, start=1):
    part = df[df["setup"] == setup]
    fig.add_trace(
        go.Scatter(
            x=part["true_outcome"],
            y=part["pred_outcome"],
            mode="markers",
            marker={
                "size": 6,
                "opacity": 0.45,
                "color": part["parameter_value"],
                "colorscale": "Viridis",
                "showscale": col == len(setups),
                "colorbar": {"title": "parameter"},
            },
            name=setup,
            showlegend=False,
        ),
        row=1,
        col=col,
    )
    fig.add_trace(
        go.Scatter(x=[-1, 1], y=[-1, 1], mode="lines", line={"dash": "dash", "color": "black"}, showlegend=False),
        row=1,
        col=col,
    )
    fig.update_xaxes(title_text="true (sc2helper)", row=1, col=col, range=[-1, 1])
    fig.update_yaxes(title_text="pred (numpy)", row=1, col=col, range=[-1, 1])

fig.update_layout(height=440, width=max(1100, 320 * len(setups)), template="plotly_white")
fig.show()


In [4]:
rows = []
for (setup, parameter_value), part in df.groupby(["setup", "parameter_value"], sort=True):
    if len(part) < 2:
        pred_numpy_score = np.nan
        pred_hp_ratio_score = np.nan
    else:
        pred_numpy_score = r2_score(part["true_outcome"], part["pred_outcome"])
        pred_hp_ratio_score = r2_score(part["true_outcome"], part["pred_outcome_hp_ratio"])
    rows.append(
        {
            "setup": setup,
            "parameter_value": float(parameter_value),
            "score": 1.0,
            "source": "true",
        }
    )
    rows.append(
        {
            "setup": setup,
            "parameter_value": float(parameter_value),
            "score": float(pred_numpy_score),
            "source": "pred_numpy",
        }
    )
    rows.append(
        {
            "setup": setup,
            "parameter_value": float(parameter_value),
            "score": float(pred_hp_ratio_score),
            "source": "pred_hp_ratio",
        }
    )

score_df = pd.DataFrame(rows)
score_df.head(9)


,setup,parameter_value,score,source
0,circle,2.0,1.000000,true
1,circle,2.0,-0.077716,pred_numpy
2,circle,2.0,-0.274384,pred_hp_ratio
3,circle,4.0,1.000000,true
4,circle,4.0,0.063657,pred_numpy
5,circle,4.0,-0.174952,pred_hp_ratio
6,circle,6.0,1.000000,true
7,circle,6.0,-0.183123,pred_numpy
8,circle,6.0,-0.354270,pred_hp_ratio


In [5]:
fig = make_subplots(rows=1, cols=len(setups), subplot_titles=setups)
for col, setup in enumerate(setups, start=1):
    part = score_df[score_df["setup"] == setup].sort_values("parameter_value")
    for source, color in [("true", "#2ca02c"), ("pred_numpy", "#1f77b4"), ("pred_hp_ratio", "#ff7f0e")]:
        sub = part[part["source"] == source]
        fig.add_trace(
            go.Scatter(
                x=sub["parameter_value"],
                y=sub["score"],
                mode="lines+markers",
                name=source,
                line={"color": color},
                showlegend=(col == 1),
            ),
            row=1,
            col=col,
        )
    fig.update_xaxes(title_text="parameter", row=1, col=col)
    fig.update_yaxes(title_text="r2", row=1, col=col)

fig.update_layout(height=420, width=max(1100, 320 * len(setups)), template="plotly_white")
fig.show()
